In [1]:
import requests
import json
from datetime import datetime
from pprint import pprint
import pandas as pd
from dotenv import load_dotenv, find_dotenv
import os


load_dotenv(find_dotenv())

# https://discordapp.com/oauth2/authorize?scope=bot&permissions=109696&client_id={CLIENT_ID}

True

In [2]:
auth_token = os.environ['USER_TOKEN'] # Find user token using this guide https://pastebin.com/PLdP21j1

curl --header "Authorization: USER_TOKEN" https://discordapp.com/api/channels/425458098535137282/messages >> out.json


<br />

you will need to combine the messages 50 at a time until the returned message length is less than 50

First message returned is newest 

You must also use a user token, bot tokens are not authorized to make the needed request

In [3]:
BASE_API = "https://discordapp.com/api"

headers = {"Authorization": auth_token}
bounties_id = 425458098535137282
moderation_id = 481838545531699210

def message_endpoint(channel_id, before=''):
    if before:
        return f"{BASE_API}/channels/{channel_id}/messages?before={before}"
    return f"{BASE_API}/channels/{channel_id}/messages"


def get_all_messages(channel_id, headers=''):
    """
    Grabs all the messages from a discord channel
    """
    data = []
    bounties_msg_endpoint = message_endpoint(channel_id)
    req = requests.get(bounties_msg_endpoint, headers=headers)
    pull = req.json()
    data = data + pull
    while len(pull) == 50:
        try:
            before = data[-1]['id']
        except IndexError:
            before = ''
        bounties_msg_endpoint = message_endpoint(channel_id, before=before)
        req = requests.get(bounties_msg_endpoint, headers=headers)
        pull = req.json()
        data = data + pull
    return data


In [4]:
bounties_messages = get_all_messages(bounties_id, headers=headers)

In [5]:
with open(f'outfiles/bounties_messages_{datetime.now()}.json', 'w') as outfile:
    json.dump(bounties_messages, outfile)

In [6]:
bounties_messages[0]

{'attachments': [],
 'tts': False,
 'embeds': [{'url': 'https://paper.swapblocks.io/.',
   'type': 'link',
   'description': 'Paper wallet generator for SBX cryptocurrency.',
   'title': 'SBX Paper Wallet'}],
 'timestamp': '2018-12-10T23:57:21.625000+00:00',
 'mention_everyone': False,
 'id': '521837862191235102',
 'pinned': False,
 'edited_timestamp': None,
 'author': {'username': 'swapblocks_lance',
  'discriminator': '1762',
  'id': '385807070299160597',
  'avatar': '3e0ede122ac47c7f8e7e0e315df4d833'},
 'mention_roles': [],
 'content': 'I still need SBX address from everyone mentioned in this message.  \n\nIf your name is below please DM me an sbx address.\n\nhttps://paper.swapblocks.io\n\n<@406112271794962433> still missing\n<@415822454787932162> still missing\n<@452463560413675522> still missing\n<@498869778061066240> still missing\n<@500292246046769152> still missing\n<@508878619951628298> still missing\n<@512954663557922818> still missing\n<@516910535514521619> still missing',
 

In [7]:
moderation_messages = get_all_messages(moderation_id, headers=headers)

In [8]:
with open(f'outfiles/moderation_messages_{datetime.now()}.json', 'w') as outfile:
    json.dump(moderation_messages, outfile)

In [9]:
# merge bounties and moderation messages into one
bounties_messages = bounties_messages + moderation_messages

## Clean Messages

For bounty payouts, I only need the username, timestamp and sbx addresses


1. First filter should only keep: author and content
- Second filter should remove all records not containing an SBX address
- Add userid and username to data
- Remove author

In [10]:
# First filter keeps author, content and timestamp
first_b_clean = []
for msg in bounties_messages:
    first_b_clean.append({k: msg[k] for k in ('author', 'content')})

In [11]:
pprint(first_b_clean[0])

{'author': {'avatar': '3e0ede122ac47c7f8e7e0e315df4d833',
            'discriminator': '1762',
            'id': '385807070299160597',
            'username': 'swapblocks_lance'},
 'content': 'I still need SBX address from everyone mentioned in this '
            'message.  \n'
            '\n'
            'If your name is below please DM me an sbx address.\n'
            '\n'
            'https://paper.swapblocks.io\n'
            '\n'
            '<@406112271794962433> still missing\n'
            '<@415822454787932162> still missing\n'
            '<@452463560413675522> still missing\n'
            '<@498869778061066240> still missing\n'
            '<@500292246046769152> still missing\n'
            '<@508878619951628298> still missing\n'
            '<@512954663557922818> still missing\n'
            '<@516910535514521619> still missing'}


In [12]:
# Second filter removes all messages that are not SBX Addresses
second_b_clean = []
for msg in first_b_clean:
    if (len(msg['content']) == 34) and (msg['content'][0] == 'X'):
        msg['userid'] = msg['author']['id']
        msg['username'] = f"{msg['author']['username']}#{msg['author']['discriminator']}"
        second_b_clean.append(msg)

In [13]:
pprint(second_b_clean[0])

{'author': {'avatar': None,
            'discriminator': '4993',
            'id': '459303069935730689',
            'username': 'JALALCCO'},
 'content': 'XHUqLsQ5qEnhLhXFt8uYq4NU8mNBJRosHX',
 'userid': '459303069935730689',
 'username': 'JALALCCO#4993'}


In [14]:
# First filter keeps author, content and timestamp
third_b_clean = []
for msg in second_b_clean:
    third_b_clean.append({k: msg[k] for k in ('content', 'userid', 'username')})

In [15]:
third_b_clean[0]

{'content': 'XHUqLsQ5qEnhLhXFt8uYq4NU8mNBJRosHX',
 'userid': '459303069935730689',
 'username': 'JALALCCO#4993'}

In [16]:
bounty_addrs = third_b_clean

## Users and roles

Next step is to collect all the users and their roles to determine who gets a bounty.  Filter out all users who do not have one of the below roles

- @five_invits 1000
- @ten_invites 5000
- @fifty_invites 25000
- @hundred_invites 50000
- @thousand_invites 500000

### Api Call

curl --header "Authorization: Mzg1ODA3MDcwMjk5MTYwNTk3.Dl4-mQ.pyzmAzJRlvhlGXrNXpdPfg-BbRk" https://discordapp.com/api/guilds/425456443412447243/members?limit=1000

In [17]:
SBX_GUILD_ID = 425456443412447243


def members_endpoint(guild_id, after=''):
    if after:
        return f"{BASE_API}/guilds/{guild_id}/members?limit=1000&after={after}"
    return f"{BASE_API}/guilds/{guild_id}/members?limit=1000"


def get_all_members(guild_id, headers=''):
    """
    Grabs all the messages from a discord channel
    """
    try:
        data = []
        req = requests.get(members_endpoint(guild_id), headers=headers)
        pull = req.json()
        data = data + pull
        while len(pull) == 1000:
            try:
                after = data[-1]['user']['id']
            except IndexError:
                after = ''
            members_url = members_endpoint(guild_id, after=after)
            req = requests.get(members_url, headers=headers)
            pull = req.json()
            data = data + pull
    except TypeError:
        print(pull)
        print(members_url)
    return data

In [18]:
members = get_all_members(SBX_GUILD_ID, headers=headers)

In [19]:
members[-1]

{'nick': None,
 'user': {'username': 'mili2010',
  'discriminator': '7686',
  'id': '521790709364424704',
  'avatar': None},
 'roles': [],
 'mute': False,
 'deaf': False,
 'joined_at': '2018-12-10T20:50:11.228000+00:00'}

In [20]:
with open(f'outfiles/sbx_members_{datetime.now()}.json', 'w') as outfile:
    json.dump(bounties_messages, outfile)

In [21]:
members_with_roles = []
bots = []

for member in members:
    if len(member['roles']) > 0:
        members_with_roles.append(member)
    try:
        if member['user']['bot']:
            bots.append(member)
    except KeyError:
        continue

## Get role ids

In [22]:
req = requests.get("https://discordapp.com/api/guilds/425456443412447243/roles", headers=headers)
roles = req.json()
roles_dict = {}
for r in roles:
    roles_dict[r['id']] = r['name']
pprint(roles_dict)

{'425456443412447243': '@everyone',
 '425463291335147528': '@SwapBlocksTeam',
 '425463946753867778': '@admin',
 '474299760409444352': '@forging_delegate',
 '481644253752328192': 'MEE6',
 '481901573019009025': 'InviteManager',
 '481931549668016170': '@fiveInvites',
 '481931920977166336': '@tenInvites',
 '481931994557710337': '@fiftyInvites',
 '481932067454713887': '@hundredInvites',
 '481933094924976139': '@1000Invites',
 '518572716618678274': 'DataBot',
 '518589428323581953': 'Altpocket',
 '521417905427185674': 'Delegate Info Bot'}


In [23]:
FIVE_INVITES_ID = "481931549668016170"
TEN_INVITES_ID = "481931920977166336"
FIFTY_INVITES_ID = "481931994557710337"
HUNDRED_INVITES_ID = "481932067454713887"
THOUSAND_INVITES_ID = "481933094924976139"

VALID_ROLES_LIST = [FIVE_INVITES_ID, TEN_INVITES_ID, FIFTY_INVITES_ID, 
              HUNDRED_INVITES_ID, THOUSAND_INVITES_ID]

VALID_ROLES_DICT = {
        "481931549668016170": "FIVE_INVITES_ID",
        "481931920977166336": "TEN_INVITES_ID",
        "481931994557710337": "FIFTY_INVITES_ID",
        "481932067454713887": "HUNDRED_INVITES_ID",
        "481933094924976139": "THOUSAND_INVITES_ID"
    }

## Limit list to invite bounty members

In [24]:
invite_bnty_members = []
other_members = []
invalid_roles = []
for member in members_with_roles:
    if len(set(member['roles']).intersection(set(VALID_ROLES_LIST))) > 0:
        invite_bnty_members.append(member)
    else:
        invalid_roles.append(member['roles'])
        other_members.append(member)

In [25]:
# Check count to see if it sounds right
len(invite_bnty_members)

304

In [26]:
# Check roles left over to ensure all eligble members receive bounty payout
x = []
for y in invalid_roles:
    for a in y:
        x.append(a)
for x in set(x):
    print(roles_dict[x])

@admin
Delegate Info Bot
@SwapBlocksTeam
DataBot
@forging_delegate
InviteManager
Altpocket
MEE6


In [27]:
pprint(invite_bnty_members[0])

{'deaf': False,
 'joined_at': '2018-11-21T07:15:56.248000+00:00',
 'mute': False,
 'nick': None,
 'roles': ['481931549668016170', '481931920977166336'],
 'user': {'avatar': 'c083108e0ca95be31a2ce18373fa1dae',
          'discriminator': '2601',
          'id': '124868289791918080',
          'username': 'Nakkal'}}


In [28]:
# Add userid and username column to invite_bnty_members

for msg in invite_bnty_members:
    msg['userid'] = msg['user']['id']
    msg['username'] = f"{msg['user']['username']}#{msg['user']['discriminator']}"

In [29]:
invite_bnty_members[0]

{'nick': None,
 'user': {'username': 'Nakkal',
  'discriminator': '2601',
  'id': '124868289791918080',
  'avatar': 'c083108e0ca95be31a2ce18373fa1dae'},
 'roles': ['481931549668016170', '481931920977166336'],
 'mute': False,
 'deaf': False,
 'joined_at': '2018-11-21T07:15:56.248000+00:00',
 'userid': '124868289791918080',
 'username': 'Nakkal#2601'}

In [30]:
bounty_addrs[-1]

{'content': 'XYdq5q3dkAm12yxnALUswT773DMy9bpnpJ',
 'userid': '425923818201153537',
 'username': 'sranbirlove#5744'}

## Add address to invite_bnty_members

In [31]:
# Add address to invite_bnty_members
for member in invite_bnty_members:
    user = member['userid']
    for addr in bounty_addrs:
        if addr['userid']  == user:
            member['address'] = addr['content']

In [32]:
invite_bnty_members[0]

{'nick': None,
 'user': {'username': 'Nakkal',
  'discriminator': '2601',
  'id': '124868289791918080',
  'avatar': 'c083108e0ca95be31a2ce18373fa1dae'},
 'roles': ['481931549668016170', '481931920977166336'],
 'mute': False,
 'deaf': False,
 'joined_at': '2018-11-21T07:15:56.248000+00:00',
 'userid': '124868289791918080',
 'username': 'Nakkal#2601',
 'address': 'XFpiuYhksm5ZUiySL8oBuiAgbs2eZ4LUvV'}

## Add role count

<p>
1 = 5 invites <br />
2 = 10 invites <br />
3 = 50 invites <br />
4 = 100 invites <br />
5 = 1000 invites <br />
</p>

In [33]:
for member in invite_bnty_members:
    member['rolecount'] = len(set(member['roles']).intersection(set(VALID_ROLES_LIST)))

In [34]:
invite_bnty_members[0]

{'nick': None,
 'user': {'username': 'Nakkal',
  'discriminator': '2601',
  'id': '124868289791918080',
  'avatar': 'c083108e0ca95be31a2ce18373fa1dae'},
 'roles': ['481931549668016170', '481931920977166336'],
 'mute': False,
 'deaf': False,
 'joined_at': '2018-11-21T07:15:56.248000+00:00',
 'userid': '124868289791918080',
 'username': 'Nakkal#2601',
 'address': 'XFpiuYhksm5ZUiySL8oBuiAgbs2eZ4LUvV',
 'rolecount': 2}

## Build csv transactions

Format:

<pre>
XKyg87BUjmxNE2URgKmGZNFBeZn9wK5MH7,5000,barm79 10 invites bounty
</pre>

In [35]:
try:
    os.remove("cleaned_payout_data/transactions.csv")
except OSError:
    pass

role_key = {
    1: ("5 invites", 1000),
    2: ("10 invites", 5000),
    3: ("50 invites", 25000),
    4: ("100 invites", 50000),
    5: ("1000 invites", 500000)
    }

for member in invite_bnty_members:
    try:
        address = member['address']
    except KeyError:
        address = ""

    rolec = member['rolecount']
    row = f"{address},{role_key[rolec][1]},{member['username']} {role_key[rolec][0]} invites bounty\n"
    with open('cleaned_payout_data/transactions.csv','a+') as fd:
        fd.write(row)

In [36]:
payout_trns = pd.read_csv('cleaned_payout_data/transactions.csv', names=["Address", "Amount", "Message"])

In [37]:
payout_trns.head()

,Address,Amount,Message
0,XFpiuYhksm5ZUiySL8oBuiAgbs2eZ4LUvV,5000,Nakkal#2601 10 invites invites bounty
1,XTcVkde4XqzkZui4c7AgMoes45wCTDTML2,1000,Deathshoot10#4468 5 invites invites bounty
2,XExppB4urDZkYHMprPCoNsLNuRxYVwRFfw,1000,IrenOK0803#7518 5 invites invites bounty
3,XLyq7gyydLUYts6d1qdyPFHSYUP5YLmX16,5000,yingchun#7343 10 invites invites bounty
4,NaN,1000,PNWDrew#3035 5 invites invites bounty


In [38]:
payout_trns.loc[17, 'Address'] = "XExUNYHbjq5SpE22i96MKSrtxVch1MxpnF" #bluedragon

In [39]:
no_address = payout_trns[payout_trns['Address'].isnull()].copy()
no_address.head()

,Address,Amount,Message
4,NaN,1000,PNWDrew#3035 5 invites invites bounty
8,NaN,50000,Thuyên Trần#8248 100 invites invites bounty
20,NaN,1000,swapblocks_lance#1762 5 invites invites bounty
34,NaN,25000,Airdrop Alert Daily YouTube#4047 50 invites in...
50,NaN,1000,Phongag#5805 5 invites invites bounty


In [40]:
"""
bluedragon 5 invites bounty
si Nakkal 5 invites bounty
kucingone 5 invites bounty
barm79 5 invites bounty
PNWDrew 5 invites bounty
Tuning 5 invites bounty
si Nakkal 10 invites bounty
barm79 10 invites bounty
"""
remove_users = ['swapblocks_lance#1762 5 invites invites bounty', 'PNWDrew#3035 5 invites invites bounty']

# index 4,19

In [41]:
no_address.to_csv('cleaned_payout_data/noaddress.csv', index=False)

In [42]:
payout_trns = payout_trns[payout_trns['Address'].notnull()].sort_values('Amount', ascending=False).reset_index(drop=True)

In [43]:
payout_trns.to_csv('cleaned_payout_data/transactions_full.csv', index=False)

## After some manually cleaning I am going to import the dataset for one last verification

In [44]:
df = pd.read_csv('cleaned_payout_data/transactions_trimmed.csv')

In [45]:
df['Amount'].sum()

1458000